In [1]:
import json
from imslp_scraping import get_all_composer_pieces, get_composer_url
from pieces import create_piece
from ydata_profiling import ProfileReport
import pyarrow

import pandas as pd
import numpy as np
import polars as pl
def piece_to_dict(piece):
    piece_dict = vars(piece).copy()
    piece_dict["movements"] = json.dumps([vars(m) for m in piece.movements])
    return piece_dict


/Users/michaelbrandt/personal/PracticeLogger/data_ingest/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
composers = ["Beethoven, Ludwig van", "Chopin, Frédéric"]
url = get_composer_url(composers[1])
data = get_all_composer_pieces(url)
pieces = []
for piece_url in data[:100]:
    piece = create_piece(url=piece_url)
    print(piece)
    pieces.append(piece)


Piece(work_name='Scherzo No.1', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.20', catalogue_type='Op', catalogue_number=20, catalogue_number_secondary=None, catalogue_id=None, composition_year=1833, composition_year_string='1833', key_signature='bminor', movements=[], sub_piece_type=None, sub_piece_count=None, instrumentation=['piano'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Scherzo_No.1%2C_Op.20_(Chopin%2C_Fr%C3%A9d%C3%A9ric)', wikipedia_url='http://en.wikipedia.org/wiki/Scherzo_No._1_(Chopin)')
Piece(work_name='Variations sur un air national allemande', composer_name='Chopin, Frédéric', catalogue_desc_str='B.14', catalogue_type='B', catalogue_number=14, catalogue_number_secondary=None, catalogue_id=None, composition_year=1826, composition_year_string='1826-27', key_signature='e', movements=[], sub_piece_type=None, sub_piece_count=None, instrumentation=['piano'], nickname='Introduction and Variation on the German Air "Der Schweizerbub"', p

Piece(work_name='Mazurkas', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.24', catalogue_type='Op', catalogue_number=24, catalogue_number_secondary=None, catalogue_id=None, composition_year=1834, composition_year_string='1834-35', key_signature=None, movements=[Movement(title='Lento', number=1, key_signature='gminor', download_url='https://imslp.org/wiki/Special:ImagefromIndex/859581'), Movement(title='Allegro non troppo', number=2, key_signature='c', download_url='https://imslp.org/wiki/Special:ImagefromIndex/859582'), Movement(title='Moderato con anima', number=3, key_signature='aflat', download_url='https://imslp.org/wiki/Special:ImagefromIndex/859583'), Movement(title='Moderato', number=4, key_signature='bflatminor', download_url='https://imslp.org/wiki/Special:ImagefromIndex/859583')], sub_piece_type='pieces', sub_piece_count=4, instrumentation=['piano'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Mazurkas%2C_Op.24_(Chopin%2C_Fr%C3%A9d%C3%A

Piece(work_name='Mazurkas', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.7\xa0; CT 55-59', catalogue_type='Op', catalogue_number=7, catalogue_number_secondary=None, catalogue_id=None, composition_year=1824, composition_year_string='1824-31', key_signature=None, movements=[Movement(title='Vivace', number=1, key_signature='bflat', download_url=None), Movement(title='Vivo, ma non troppo', number=2, key_signature='aminor', download_url=None), Movement(title='Mazurka (no tempo indication)', number=3, key_signature='fminor', download_url=None), Movement(title='Presto', number=4, key_signature='aflat', download_url=None), Movement(title='Vivo ,senza fine*', number=5, key_signature='c', download_url=None), Movement(title='Vivace', number=1, key_signature=None, download_url=None)], sub_piece_type='pieces', sub_piece_count=5, instrumentation=['piano'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Mazurkas%2C_Op.7_(Chopin%2C_Fr%C3%A9d%C3%A9ric)', wikipedia_

Piece(work_name='Verschiedene Werke', composer_name='Chopin, Frédéric', catalogue_desc_str=None, catalogue_type=None, catalogue_number=None, catalogue_number_secondary=None, catalogue_id=None, composition_year=None, composition_year_string=None, key_signature=None, movements=[Movement(title='Marche funebre', number=1, key_signature=None, download_url=None), Movement(title='Polonaise', number=2, key_signature=None, download_url=None), Movement(title='Polonaise', number=3, key_signature=None, download_url=None), Movement(title='Andante spianato', number=4, key_signature=None, download_url=None), Movement(title='Pr', number=5, key_signature=None, download_url=None), Movement(title='Pr', number=6, key_signature=None, download_url=None), Movement(title='Pr', number=7, key_signature=None, download_url=None), Movement(title='Berceuse', number=8, key_signature=None, download_url=None), Movement(title='Andantino', number=9, key_signature=None, download_url=None), Movement(title='Largo', number=

Piece(work_name='Piano Concerto No.2', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.21', catalogue_type='Op', catalogue_number=21, catalogue_number_secondary=None, catalogue_id=None, composition_year=1829, composition_year_string='1829-30', key_signature='fminor', movements=[Movement(title='Maestoso', number=1, key_signature='fminor', download_url='https://imslp.org/wiki/Special:ImagefromIndex/55548'), Movement(title='Larghetto', number=2, key_signature='aflat', download_url='https://imslp.org/wiki/Special:ImagefromIndex/55549'), Movement(title='Allegro vivace', number=3, key_signature='fminor', download_url='https://imslp.org/wiki/Special:ImagefromIndex/55550'), Movement(title='Orchestra,Frédéric Chopin(piano)', number=1, key_signature=None, download_url=None)], sub_piece_type='movements', sub_piece_count=3, instrumentation=['piano', 'orchestra'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Piano_Concerto_No.2%2C_Op.21_(Chopin%2C_Fr%C3%A9d%C3%A

...

Piece(work_name='Etudes', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.10', catalogue_type='Op', catalogue_number=10, catalogue_number_secondary=None, catalogue_id=None, composition_year=1829, composition_year_string='1829-32', key_signature=None, movements=[Movement(title='Etude in C major', number=1, key_signature=None, download_url=None), Movement(title='Etude in A minor', number=2, key_signature=None, download_url=None), Movement(title='Etude in E major', number=3, key_signature=None, download_url=None), Movement(title='Etude in C', number=4, key_signature=None, download_url=None), Movement(title='Etude in G', number=5, key_signature=None, download_url=None), Movement(title='Etude in E', number=6, key_signature=None, download_url=None), Movement(title='Etude in C major', number=7, key_signature=None, download_url=None), Movement(title='Etude in F major', number=8, key_signature=None, download_url=None), Movement(title='Etude in F minor', number=9, key_signature=None, do

Piece(work_name='Mazurkas', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.68 (Posthumous)', catalogue_type='Op', catalogue_number=68, catalogue_number_secondary=None, catalogue_id=None, composition_year=1826, composition_year_string='1826-1849', key_signature='e', movements=[Movement(title='Vivace', number=1, key_signature='c', download_url=None), Movement(title='Lento', number=2, key_signature='aminor', download_url=None), Movement(title='Allegro ma non troppo', number=3, key_signature='f', download_url=None), Movement(title='Andantino', number=4, key_signature='fminor', download_url=None)], sub_piece_type='mazurkas', sub_piece_count=4, instrumentation=['piano'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Mazurkas%2C_Op.68_(Chopin%2C_Fr%C3%A9d%C3%A9ric)', wikipedia_url='http://en.wikipedia.org/wiki/Mazurkas,_Op._68_(Chopin)')
Piece(work_name='Rondo', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.16', catalogue_type='Op', catalogue_nu

Piece(work_name='Douze Etudes', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.25', catalogue_type='Op', catalogue_number=25, catalogue_number_secondary=None, catalogue_id=None, composition_year=1832, composition_year_string='1832-36', key_signature=None, movements=[Movement(title='Etude in A', number=1, key_signature=None, download_url=None), Movement(title='Etude in F minor', number=2, key_signature=None, download_url=None), Movement(title='Etude in F major', number=3, key_signature=None, download_url=None), Movement(title='Etude in A minor', number=4, key_signature=None, download_url=None), Movement(title='Etude in E minor', number=5, key_signature=None, download_url=None), Movement(title='Etude in G', number=6, key_signature=None, download_url=None), Movement(title='Etude in C', number=7, key_signature=None, download_url=None), Movement(title='Etude in D', number=8, key_signature=None, download_url=None), Movement(title='Etude in G', number=9, key_signature=None, download

Piece(work_name='Scherzo No.3', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.39', catalogue_type='Op', catalogue_number=39, catalogue_number_secondary=None, catalogue_id=None, composition_year=1839, composition_year_string='1839', key_signature='csharpminor', movements=[], sub_piece_type=None, sub_piece_count=None, instrumentation=['piano'], nickname=None, piece_style='romantic', imslp_url='https://imslp.org/wiki/Scherzo_No.3%2C_Op.39_(Chopin%2C_Fr%C3%A9d%C3%A9ric)', wikipedia_url='http://en.wikipedia.org/wiki/Scherzo_No._3_(Chopin)')
Piece(work_name='Waltz', composer_name='Chopin, Frédéric', catalogue_desc_str=None, catalogue_type=None, catalogue_number=None, catalogue_number_secondary=None, catalogue_id=None, composition_year=1824, composition_year_string='1824-49 ca.', key_signature='aminor', movements=[], sub_piece_type='waltz', sub_piece_count=1, instrumentation=['piano'], nickname='Valse', piece_style='romantic', imslp_url='https://imslp.org/wiki/Waltz_in_A_minor_(Cho

Piece(work_name='17 Polish Songs', composer_name='Chopin, Frédéric', catalogue_desc_str='Op.74', catalogue_type='Op', catalogue_number=74, catalogue_number_secondary=None, catalogue_id=None, composition_year=1829, composition_year_string='1829-1847', key_signature=None, movements=[Movement(title='Życzenie.Allegro ma non troppo Narodowy Instytut Fryderyka Chopina;All Music Guide(1829)', number=1, key_signature='g', download_url=None), Movement(title='Wiosna.Andantino Narodowy Instytut Fryderyka Chopina;All Music Guide(1838)', number=2, key_signature='gminor', download_url=None), Movement(title='Smutna rzeka.Allegretto Narodowy Instytut Fryderyka Chopina;All Music Guide(1831)', number=3, key_signature='fsharpminor', download_url=None), Movement(title='Hulanka.Vivace Narodowy Instytut Fryderyka Chopina;All Music Guide(1830)', number=4, key_signature='c', download_url=None), Movement(title='Gdzie lubi.Allegro Narodowy Instytut Fryderyka Chopina;All Music Guide(1829)', number=5, key_signatu

In [3]:
pieces_dict = [piece_to_dict(piece) for piece in pieces] 
df = pl.DataFrame(pieces_dict, strict=False)
print(df)
df.head(100)

shape: (100, 18)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ work_name ┆ composer_ ┆ catalogue ┆ catalogue ┆ … ┆ nickname  ┆ piece_sty ┆ imslp_url ┆ wikipedi │
│ ---       ┆ name      ┆ _desc_str ┆ _type     ┆   ┆ ---       ┆ le        ┆ ---       ┆ a_url    │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ ---       ┆ str       ┆ ---      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆           ┆ str       ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Scherzo   ┆ Chopin,   ┆ Op.20     ┆ Op        ┆ … ┆ null      ┆ romantic  ┆ https://i ┆ http://e │
│ No.1      ┆ Frédéric  ┆           ┆           ┆   ┆           ┆           ┆ mslp.org/ ┆ n.wikipe │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ wiki/Sche ┆ dia.org/ │
│           ┆           ┆           ┆           ┆   ┆           ┆         

work_name,composer_name,catalogue_desc_str,catalogue_type,catalogue_number,catalogue_number_secondary,catalogue_id,composition_year,composition_year_string,key_signature,movements,sub_piece_type,sub_piece_count,instrumentation,nickname,piece_style,imslp_url,wikipedia_url
str,str,str,str,i64,i64,null,i64,str,str,str,str,i64,list[str],str,str,str,str
"""Scherzo No.1""","""Chopin, Frédéric""","""Op.20""","""Op""",20,null,null,1833,"""1833""","""bminor""","""[]""",null,null,"[""piano""]",null,"""romantic""","""https://imslp.org/wiki/Scherzo…","""http://en.wikipedia.org/wiki/S…"
"""Variations sur un air national…","""Chopin, Frédéric""","""B.14""","""B""",14,null,null,1826,"""1826-27""","""e""","""[]""",null,null,"[""piano""]","""Introduction and Variation on …","""romantic""","""https://imslp.org/wiki/Variati…",null
"""Rondo""","""Chopin, Frédéric""","""Op.1""","""Op""",1,null,null,1825,"""1825""","""cminor""","""[]""",null,null,"[""piano""]","""Adieu à Varsovie""","""romantic""","""https://imslp.org/wiki/Rondo_i…","""http://en.wikipedia.org/wiki/R…"
"""Polonaise""","""Chopin, Frédéric""","""Op.53""","""Op""",53,null,null,1842,"""1842""","""aflat""","""[]""",null,null,"[""piano""]","""Héroique Polonaise""","""romantic""","""https://imslp.org/wiki/Polonai…","""http://en.wikipedia.org/wiki/P…"
"""Rondo""","""Chopin, Frédéric""","""Op.73""","""Op""",73,null,null,1828,"""1828""","""c""","""[]""",null,null,"[""2 pianos""]",null,"""romantic""","""https://imslp.org/wiki/Rondo_i…","""http://en.wikipedia.org/wiki/R…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Lieder""","""Chopin, Frédéric""",null,null,null,null,null,null,null,null,"""[]""",null,null,"[""voice"", ""piano""]",null,"""romantic""","""https://imslp.org/wiki/Lieder_…",null
"""Polonaise""","""Chopin, Frédéric""","""Op.44""","""Op""",44,null,null,1841,"""1841""","""fsharpminor""","""[]""",null,null,"[""piano""]",null,"""romantic""","""https://imslp.org/wiki/Polonai…","""http://en.wikipedia.org/wiki/P…"
"""Variations on 'Là ci darem la …","""Chopin, Frédéric""","""Op.2""","""Op""",2,null,null,1827,"""1827""","""bflat""","""[{""title"": ""Introduction. Larg…","""sections""",7,"[""piano"", ""orchestra""]",null,"""romantic""","""https://imslp.org/wiki/Variati…","""http://en.wikipedia.org/wiki/V…"


In [4]:
profile = ProfileReport(df.to_pandas(), title="Pandas Profiling Report")

profile.to_file('data_summary.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/michaelbrandt/personal/PracticeLogger/data_ingest/.venv/lib/python3.12/site-packages/ydata_profiling/model/pandas/summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 463.00it/s]
